# 第11回 その2: 任意の中間層数，ノード数を設定可能なニューラルネットワークの実装
`11_01_neural_network.ipynb`では任意のノード数を設定できますが，中間層の数が1に限定されています。  
ここでは，中間層の数を任意に設定できるように，さらに実装を改良します。     

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/11"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/11/
# 現在のフォルダの中身を表示
%ls

`2class_data_circle.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data_circle.csv` を読み込みます。  
このデータはx1とx2の二次元データで，クラス0とクラス1の2クラスのどちらかに属しています。  
入力Xの標準化やクラスYのone-hotベクトルの作成も，ここで行っておきます。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data_circle.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# one-hotベクトルの作成。
Y_onehot = np.zeros([num_samples, num_classes])
for n in range(num_samples):
  # クラス番号(y[n])に対応するインデクスを1にする
  Y_onehot[n, Y[n]] = 1

color_list = [(0, 0, 1), (1, 0, 0)]
# データをプロット
plt.figure(figsize=(7,7))
for k in range(num_classes):
  # n番目のクラスに属している(Y==c)データをプロット
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()


## ステップ2: 中間層数が可変のニューラルネットワークの実装  
まず，シグモイド関数やソフトマックス関数，クロスエントロピー計算の関数を定義しておきます。

In [ ]:
def sigmoid(z):
  '''
      シグモイド関数
  '''
  return (1+np.exp(-1*z))**(-1)


def softmax(z):
  '''
      ソフトマックス関数
      z: 要素数=クラス数のベクトル
      y_hat: クラス毎の確率。要素数=クラス数のベクトル
  '''
  z_norm = z - np.max(z) # np.exp(z)のオーバーフロー対策
  return (np.exp(z_norm) / np.sum(np.exp(z_norm)))


def cross_entropy(y_hat, y_onehot):
  '''
      クロスエントロピー損失を計算する。
      y_hat: マルチクラス回帰によって推定された y
      y_onehot: 正解のクラスの確率が1，それ以外の確率を0とするベクトル
                (one-hotベクトル)
  '''
  return -1.0 * np.sum(y_onehot*np.log(y_hat))

ここから，中間層の数および各中間層のノード数を可変とするニューラルネットワークを実装していきます。  
まず，以下の関数`initialize`では，引数で指定した中間層数とノード数を反映した，中間層および出力層の重み行列の初期値を作成します。

ここでは，変数`num_mid`をリストとして定義することで，中間層の数および各中間層のノード数を定義できるようにしています。  
`num_mid = [3]`の場合，「ノード数3の中間層を1個」という意味です。  
`num_mid = [4, 3]`の場合，「中間層は2個で，一つ目の中間層のノード数は4，二つ目の中間層のノード数は3」という意味です。  
`num_mid = []`の場合，「中間層は無し」となります。  

In [ ]:
def initialize(num_inp, num_out, num_mid, seed=0):
  '''
      初期設定
      num_inp: 入力層のノード数(データの次元数)
      num_out: 出力層のノード数(=識別したいクラス数)
      num_mid: 各中間層のノード数。リスト型で定義
               中間層が2層で，それぞれノード数が5, 3とする場合
               num_mid = [5, 3]とする。
      seed: 乱数のシード。デフォルト値は0
  '''
  # 中間層の数は num_mid の要素数
  num_mid_layers = len(num_mid)
  # トータルの層数 = 中間層の数 + 1 (出力層の分)
  num_layers = num_mid_layers + 1

  np.random.seed(seed) # 乱数シードの固定

  # 中間層の重み行列 W の設定（各層の重み行列がリスト型で格納）
  W = []
  b = []
  for layer in range(num_layers):
    # まず，各層への入力ノード数と出力ノード数を確認
    if layer == 0:
      # 最初の中間層は入力データが入るため，入力ノード数=num_inp(データの次元数)
      n_in = num_inp
    else:
      # それ以外の場合，入力ノード数=一つ前の中間層のノード数
      n_in = num_mid[layer-1]
    if layer == num_layers - 1:
      # 出力層の場合，出力ノード数=num_out(クラス数)
      n_out = num_out
    else:
      # それ以外の場合，出力ノード数=中間層のノード数
      n_out = num_mid[layer]
      
    # W のサイズは[出力ノード数x入力ノード数]．乱数に基づいて初期値を作成
    W_tmp = np.random.normal(0, 1/np.sqrt(n_in), (n_out, n_in))
    # b のサイズは[出力ノード数]
    b_tmp = np.zeros(n_out)

    # 重み行列リストに追加
    W.append(W_tmp)
    b.append(b_tmp)

  return W, b

ここでは，各層における重み行列をリストで定義しています。  
リストには，中間層と出力層の重み行列が格納されます。  
中間層が2層存在する場合，`W = [ 一つ目の中間層のW, 二つ目の中間層のW, 出力層のW]`となります。  
`b`や勾配も同様です。

次に，ニューラルネットワークにデータを入力し，計算を実行する（フォワード処理）の関数を実装します。

In [ ]:
def forward(x, W, b):
  '''
      ニューラルネットワークにデータを通す(フォワード処理)
      x: 入力データ(サイズ=num_inpのベクトル)
      W: 各中間層および出力層の重み行列をリスト化したもの
      b: 各中間層および出力層のバイアスベクトルをリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)

  # 層に通す。
  # このとき，各層における活性化関数に通す前と通した後の計算結果をそれぞれ
  # g, h としてリスト形式で記録しておく
  g = [] # 非線形関数(活性化関数)を通す前
  h = [] # 活性化関数を通した後
  for layer in range(num_layers):
    if layer == 0:
      # 最初の中間層では，入力層の値(=入力データ x)との内積を取る
      g_tmp = np.dot(W[layer], x) + b[layer]
    else:
      # 以降の層では，一つ前の層の値(h[-1])との内積を取る
      g_tmp = np.dot(W[layer], h[-1]) + b[layer]
      
    if layer == num_layers - 1:
      # 出力層の場合はソフトマックス関数に通す
        h_tmp = softmax(g_tmp)
    else:
      # それ以外の場合はシグモイド関数に通す
        h_tmp = sigmoid(g_tmp)

    # リストに追加
    g.append(g_tmp)
    h.append(h_tmp)

  # g と h を返す。最終的な出力はh[-1]
  return g, h

勾配を計算する関数を実装します。  
ここでは，フォワード処理（`forward`関数）において計算された，各層の計算結果`h`を利用し，  
誤差逆伝播法により勾配を計算します。（バックワード処理）

In [ ]:
def backward(x, y, W, h):
  '''
      勾配を計算する(バックワード処理)
      x: 入力データ(サイズ=num_inpのベクトル)
      y: 正解の出力データ(サイズ=num_outのベクトル)
      W: 各中間層および出力層の重み行列をリスト化したもの
      h: 各中間層および出力層の計算結果をリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)
  
  # 勾配を計算
  grad_W = []
  grad_b = []
  # forのrange()の後ろの[::-1]は逆順の意味。つまりnum_layers-1～0へカウントダウン。
  for layer in range(num_layers)[::-1]:
    # まず，prev_grad = ∂L/∂g を求める。prev_gradは逆伝播される。
    # prev_gradは出力層とそれ以外で計算が異なる。
    if layer == num_layers - 1:
      # 出力層の場合
      prev_grad = h[-1] - y
    else:
      # それ以外の層の場合
      prev_grad = np.dot(prev_grad, W[layer+1])*(1 - h[layer])*h[layer]

    # prev_gradを使って勾配を計算する。
    if layer == 0:
      # 入力層の場合
      grad_W_tmp = np.dot(np.array([prev_grad]).T, np.array([x]))
    else:
      # それ以外の場合
      grad_W_tmp = np.dot(np.array([prev_grad]).T, np.array([h[layer-1]]))
    grad_b_tmp = prev_grad

    # リストの先頭に追加
    grad_W.insert(0, grad_W_tmp)
    grad_b.insert(0, grad_b_tmp)
  
  # grad_Wとgrad_bを返す
  return grad_W, grad_b

最後に，バックワード処理によって計算された勾配を使って，  
勾配降下法により重み行列を更新する関数`update`を実装します。

In [ ]:
def update(lr, W, b, grad_W, grad_b):
  '''
      勾配降下法によるパラメータの更新
      lr: 学習率
      W, b: 各中間層および出力層の重み行列とバイアスベクトルをリスト化したもの
      grad_W, grad_b，各層の勾配をリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)

  for layer in range(num_layers):
    W[layer] -= lr * grad_W[layer]
    b[layer] -= lr * grad_b[layer]
  
  return W, b

## ステップ3: ニューラルネットワークの実行
中間層の数を1，ノード数を3として分類を行います。  

In [ ]:
# 中間層のノード数
num_middle_node = [3]

# 学習率
lr = 0.2
# エポック数
num_epochs = 30

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)

np.random.seed(0)

# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

for epoch in range(num_epochs):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  Y_onehot_tmp = Y_onehot[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る
  for n in range(num_samples):
    x = X_tmp[n]
    y = Y_tmp[n]
    x = np.reshape(x, [num_dimensions])

    g, h = forward(x, W, b)
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

次に，中間層の数を2個に増やし，それぞれのノード数を3, 2として実行します。

In [ ]:
# 中間層のノード数
num_middle_node = [3, 2]

# 学習率
lr = 0.2
# エポック数
num_epochs = 30

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)

np.random.seed(0)

# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

for epoch in range(num_epochs):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  Y_onehot_tmp = Y_onehot[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る
  for n in range(num_samples):
    x = X_tmp[n]
    y = Y_tmp[n]
    x = np.reshape(x, [num_dimensions])

    g, h = forward(x, W, b)
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

こちらも100%の精度が得られました。

中間層が2のモデルでは，二つ目の中間層，つまり出力層の直前の層のノード数を2としています。  
この2個のノードの値（活性化関数を通す前の値 $g$）を2次元平面上でプロットしてみます。

In [ ]:
# 出力層の直前の層の g の値
g_tmp = np.zeros([num_samples, 2])

for n in range(num_samples):
  x = X[n]
  # ニューラルネットワークに与える
  g, h = forward(x, W, b)

  # 出力層の直前の層のインデクスは -2 
  g_tmp[n,:] = g[-2]

plt.figure(figsize=(7,7))
color_list = [(0, 0, 1), (1, 0, 0)]
for k in range(num_classes):
  plt.scatter(g_tmp[Y==k,0], g_tmp[Y==k,1], color=color_list[k], label='class'+str(k))
plt.legend()
plt.show()

元々の二次元データ$x$は線形分離不可能な分布になっていたのに対して，出力層の直前の層の値は線形分離可能な分布になっています。  

今回のニューラルネットワークモデルでは，中間層のノード数が3と2です。  
従って，元も2次元データ$x$が，最初の中間層で3次元データに変換され，さらに次の中間層で2次元データに変換されたことになります。  
この非線形な変換によって，線形分離不可能な空間から，線形分離可能な空間に変換されたことになります。

中間層の数が2の場合でも100%になりましたが，中間層の数が1の場合は2エポック目で既に100%になっていたのに対して中間層の数が2の場合では10エポック目あたりまでは50%となっており，学習スピードが遅いことが分かりました。  

そこで，試しに層の数をさらに増やして学習を実行してみましょう。

In [ ]:
# 中間層のノード数
num_middle_node = [3, 3, 3, 3]

# 学習率
lr = 0.2
# エポック数
num_epochs = 30

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)

np.random.seed(0)

# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

for epoch in range(num_epochs):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  Y_onehot_tmp = Y_onehot[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る
  for n in range(num_samples):
    x = X_tmp[n]
    y = Y_tmp[n]
    x = np.reshape(x, [num_dimensions])

    g, h = forward(x, W, b)
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

30エポックでは全く学習がされませんでした。  
各層の重み行列$\bf W$の勾配を見てみましょう。 

In [ ]:
num_layers = len(grad_W)
for n in range(num_layers):
  print('layer : '+str(n))
  print('grad_W =')
  print(grad_W[n])

層が0，つまり入力層に近づくに従って，勾配の絶対値が小さい値になっていることが分かります。  
ある層の勾配は，その後ろの層の勾配に何らかの値を掛けた値になります。  
このとき，後ろの層の勾配や，掛けられる値の絶対値が1以下の場合，その層で計算される勾配の絶対値は小さくなります。  
これが誤差逆伝播の仮定で何層も繰り返されると，入力層に近い層ほど，勾配の絶対値が小さくなっていきます。  
この現象が深刻化すると，勾配の絶対値が小さくなりすぎてアンダーフローを起こして0になってしまう，<font color='red'>**勾配消失問題**</font>が発生します。  
これは長らく深層ニューラルネットワークの学習を困難にしていた要因の一つで，この問題を軽減する様々なテクニックが近年開発されています。